In [1]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import numpy as np
import geopandas as gpd

**Genereren van de laatste 6 jaren als strings**.

haalt het huidige jaar op en genereert een lijst met de afgelopen 6 jaren.  
Dit wordt gebruikt om data te filteren op recente jaren.


In [3]:
current_year = datetime.datetime.now().year
JAREN = [str(jaar) for jaar in range(current_year - 6, current_year )]
print(JAREN)

['2019', '2020', '2021', '2022', '2023', '2024']


In [4]:
def laad_en_filter_data(pad, jaren):
    """
    Deze functie laadt een Excel-bestand in, selecteert specifieke tabbladen
    op basis van de opgegeven lijst met jaren, en combineert de data tot één Pandas DataFrame.

    Parameters:
    pad (str): Het pad naar het Excel-bestand.
    jaren (list): Een lijst met tabbladnamen (jaartallen) die moeten worden ingeladen.

    Returns:
    pd.DataFrame: Een Pandas DataFrame die de gecombineerde data van de geselecteerde tabbladen bevat.
    """
    data = pd.read_excel(pad, sheet_name= None)
    selectie_data = {jaar: data[jaar] for jaar in jaren}
    print("laden is gelukt")
    return pd.concat(selectie_data.values(),  ignore_index=True)

pad = "data/dataset vastgoed_2010_2024.xlsx"
dataset = laad_en_filter_data(pad, JAREN)


laden is gelukt


In [ ]:
dataset.head()

**Hernoemen kolommen**
  
Ik kies vaste kolomnamen en hernoem de kolomnamen via een functie.

Ik voeg een kolom toe met period type in deze functie met waarden Kwartaal, Semester en Jaar

In [ ]:
nieuwe_kolomnamen = {'CD_YEAR': 'Jaar', 
                     'CD_TYPE_NL': 'Woningtype_NL', 
                     'CD_TYPE_FR': 'Woningtype_FR', 
                     'CD_REFNIS': 'NIS-code', 
                     'CD_REFNIS_NL': 'Naam_NL', 
                     'CD_REFNIS_FR': 'Naam_FR', 
                     'CD_PERIOD': 'Periode', 
                     'MS_TOTAL_TRANSACTIONS': 'Aantal_transacties', 
                     'MS_P_25':'Prijs_25e_Percentiel', 
                     'MS_P_50_median': 'Prijs_50e_Percentiel', 
                     'MS_P_75': 'Prijs_75e_Percentiel', 
                     'CD_niveau_refnis': 'Geografisch_Niveau'}



def hernoem_kolomnamen(dataset):
    dataset.rename(columns = nieuwe_kolomnamen, inplace = True)
    dataset['periode_type'] = dataset['Periode'].map(lambda x: "Jaar" if x == 'Y' else 'Kwartaal' if x.startswith("Q") else "Semester" )
    return dataset

hernoem_kolomnamen(dataset)
dataset.head()



**Maak een bestand df om de originele dataset te behouden + verzamel informatie over de dataset**
    
maak een copy van de dataset om op te werken
    
een functie die de nodige informatie verzameld over de dataset:

- eerste 10 rijen
- info()
- output van describe in een variabel stats
- info over missing values
    
de functie retourneert ook al die waardes in variabelen voor later gebruik

Ik maak een df bestand aan en slaag de resultaten van de inspect functie op in result

In [ ]:
def create_df(dataset):
    df = dataset.copy()
    return df

def inspect_df(df):
    if df.empty:
        print("De dataset is leeg.")
        return None
    else:
        print(f"Eerste 10 rijen")
        display(df.head(10))
        print(f"Informatie over de dataset:")
        df.info()
        stats = pd.DataFrame(df.describe(include = "all"))
        missing_values = df.isnull().sum()
        missing_percentage = (missing_values/len(df)*100).round(2)
        missing_stats = pd.DataFrame({"missing_values" : missing_values, "missing_percentage": missing_percentage})
        print(f"Samenvattende statistieken:")
        display(stats)
        display(missing_stats)
        return {"stats": stats, "missing_values": missing_values, "missing_percentage": missing_percentage, "missing_stats": missing_stats}

df = create_df(dataset)
result = inspect_df(df)


**Boxplot van de numerieke kolommen**

Ik maak een boxplot van de numerieke kolommen om de spreiding van de data te bekijken en te kijken waar er outliers zijn.

In [ ]:
def create_boxplot(df, kolom):
    fig, ax = plt.subplots()
    ax.boxplot(df[kolom].dropna(), vert = False, patch_artist = True,
          boxprops = dict(facecolor = 'lightblue', color = 'black', linewidth = 2),
          medianprops = dict(color = 'red', linewidth = 2),
          whiskerprops = dict(color = 'black', linewidth = 2),
          capprops = dict(color = 'black', linewidth = 2),
          flierprops = dict(marker = 'o', color = 'red', markersize = 8))
    ax.set_title(f'Boxplot van {kolom}')
    ax.set_xlabel(f'{kolom}')
    ax.xaxis.grid(True, linestyle = '--', alpha = 0.7)
    plt.show()

numerieke_kolommen = df.select_dtypes(include = 'number').columns
print(numerieke_kolommen)
for col in numerieke_kolommen:
    create_boxplot(df, col)

In [ ]:
def check_outliers(df, kolom):
    q1 = np.percentile(df[kolom].dropna(), 25)
    q3 = np.percentile(df[kolom].dropna(), 75)
    iqr = q3 -q1
    lower_bound = q1 - 1.5 *iqr
    upper_bound = q3 + 1.5 *iqr
    outliers = df[(df[kolom] < lower_bound) | (df[kolom] > upper_bound)]
    return outliers

print(check_outliers(df, 'Prijs_25e_Percentiel').shape)
print(check_outliers(df, 'Prijs_50e_Percentiel').shape)
print(check_outliers(df, 'Prijs_75e_Percentiel').shape)

outliers_25_before = check_outliers(df, 'Prijs_25e_Percentiel')
outliers_50_before = check_outliers(df, 'Prijs_50e_Percentiel')
outliers_75_before = check_outliers(df, 'Prijs_75e_Percentiel')
print(outliers_25_before.index)
print(outliers_50_before.index)
print(outliers_75_before.index)


**Visuele voorstelling van de missing values**

In [ ]:
def visual_missing_percentage(result):
    missing_percentage = result["missing_percentage"]
    #missing_percentage_filterd = missing_percentage[missing_percentage > 0]
    #missing_percentage_filterd.plot(kind = "bar", rot = 30)
    missing_percentage.plot(kind = "bar", rot = 90)
    plt.title("Percentage Missende Waarden per Kolom")
    plt.ylabel("Percentage")
    plt.show()
visual_missing_percentage(result)



**Onderzoeken Missing Values**

We hebben informatie over de missing values in variabele result

In [ ]:
#Onderzoeken missing_values
print(result["missing_values"])

**Uitzoeken waar de missing values voorkomen**

De visual 'Percentage Missende Waarden per Kolom' toont dat de missing values zich voordoen in de kolommen Prijs_25e_Percentiel, Prijs_50e_Percentiel en Prijs_75e_Percentiel.

Groeperen op jaar en periode om na te gaan of er specifieke periodes zijn met meer missende data
- De dataset wordt gegroepeerd per Jaar en Periode.
- Het aantal missende waarden wordt per groep berekend.
- Een pivot-table wordt gebruikt om de data te structureren voor visualisatie.
- Een heatmap toont waar de ontbrekende waarden zich bevinden.



In [ ]:
def heatmap_missing_values_period_year(df):
    if "Prijs_50e_Percentiel" not in df.columns:
        print("Fout: De kolom 'Prijs_50e_Percentiel' ontbreekt in de dataset. Visual niet mogelijk")
        return
    missing_data = df.groupby(["Jaar", "Periode"])[["Prijs_25e_Percentiel", "Prijs_50e_Percentiel", "Prijs_75e_Percentiel"]].apply(lambda x : x.isnull().sum()).reset_index()
    missing_pivot = missing_data.pivot(index = "Jaar", columns = "Periode", values = "Prijs_50e_Percentiel")  
    plt.figure(figsize=(10, 6))
    sns.heatmap(missing_pivot, cmap='Reds', annot=True, fmt='g')
    plt.title("Aantal missende mediaanprijzen per Periode en jaar")
    plt.xlabel("Periode")
    plt.ylabel("Jaar")
    plt.show()

heatmap_missing_values_period_year(df)

**Missende waardes per periode type**

Vanuit de heatmap zien we dat de meeste missende waardes zich voordoen in de kwartalen en dat er voor 2024 Q4, S2 en Y helemaal geen waardes zijn.

Ik maak een groepering missing per period type om dit te controleren

De functie  berekent het aantal missende waardes per periode type.

Parameters: 
- df(de dataset)
- columns (list, optional): Lijst met kolommen waarvoor missende waarden moeten worden geanalyseerd.
                              Standaard wordt gekeken naar prijspercentielen.

Returns:
- DataFrame met aantal missende waarden per periode-type.

In [ ]:
def missing_per_periode_type(df, columns = None):
    if columns is None:
        columns = ['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']
        return df.groupby(['periode_type'])[columns].apply(lambda x: x.isnull().sum())


missing_per_periode_type_full_dataset = missing_per_periode_type(df)    
    
print(missing_per_periode_type_full_dataset)

In [ ]:
df.loc[(df["periode_type"] == "Kwartaal") & (df["Jaar"] == 2024) & (df["Naam_NL"] == "BOECHOUT"), ["Jaar","Periode","Naam_NL", "Prijs_25e_Percentiel", "Prijs_50e_Percentiel", "Prijs_75e_Percentiel"]]

**Onderzoeken of forward_fill en backward_fill een goede methode zou zijn om missende waardes te behandelen**

Waarom deze methode?
- Dataset bestaat uit mediaanprijzen dus minder gevoelig voor uitschieters
- Werken binnen gemeente, periode type, jaar en woningtype omdat prijzen daarin vaken consistent blijven
- Deze methodes zullen de missing values opvullen maar geen grote sprongen veroorzaken.

Test op gemeente Boechout in 2023 op de kwartalen



In [ ]:
missing_combinations = df.groupby(["Naam_NL", "Jaar", "Periode", "Woningtype_NL"])[["Prijs_25e_Percentiel", "Prijs_50e_Percentiel", "Prijs_75e_Percentiel"]].apply(lambda x: x.isnull().all())
missing_combinations = missing_combinations[missing_combinations].reset_index()
missing_combinations.to_excel("missing_combinations.xlsx", index=False)


In [ ]:
#Tel de missende waarden per combinatie van jaar, woningtype en periode
missing_summary = df.groupby([ 'Jaar', 'Periode', 'Woningtype_NL'])[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].apply(lambda x: x.isnull().sum())

#Toon de samenvatting
#print(missing_summary)

missing_summary.to_excel("missing_summary.xlsx")


In [ ]:
#Maak een kopie van de dataset om de originele data niet te wijzigen
df_work = df.copy()
#Optioneel: exporteer de dataset naar Excel voor inspectie
#df_work.to_excel("df_work.xlsx", index=False)

#Filter de dataset op:
#- Gemeente: "BOECHOUT"
#- Periode-type: "Kwartaal"
#- Jaar: 2023
df_filterd = df_work[
     (df_work["Naam_NL"] == "BOECHOUT") & 
     (df_work["periode_type"] == "Kwartaal") & 
     (df_work["Jaar"] == 2023)].copy()

#Optioneel: Bekijk de eerste rijen van de gefilterde dataset
#print(df_filterd)
#print(df_filterd.info())

#Controleer de index van de gefilterde dataset
print(df_filterd.index)

#Pas forward-fill en backward-fill toe op de mediaanprijzen binnen elke groep.
df_filterd.loc[:, ['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']] = (
    df_filterd.groupby(['Naam_NL', 'Jaar', 'Woningtype_NL'])[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].transform(lambda group: group.ffill().bfill()))
print(df_filterd)




**Nieuwe df_filterd met het testresultaat Boechout naar excel**

In [ ]:
df_filterd.to_excel("df_filterd.xlsx", index=False)

**Dataset beperken tot het arrondisement Mechelen + Zemst**

Ik ben alleen geïnteresseerd in het arrondissement Mechelen en de gemeente Zemst.

Daarom filter ik de dataset zodat alleen de volgende gemeenten overblijven:
- Berlaar, Bonheiden, Bornem, Duffel, Heist-op-den-Berg
- Lier, Mechelen, Nijlen, Putte, Puurs-Sint-Amands
- Sint-Katelijne-Waver, Willebroek, Zemst

Hierna bekijk ik opnieuw de dataset met de functies inspect_df en visual_missing_percentage om na te gaan of de missende waardes nu onder de 40% liggen.

Bij teveel missende waardes kan forward-fill en backward-fill verkeerde trends suggereren.

Als het percentage missende waarden onder de 40% blijft, is het opvullen statistisch betrouwbaarder.

In [ ]:
#Filteren van de dataset
gemeenten_mechelen = [
    'Berlaar', 'Bonheiden', 'Bornem', 'Duffel', 'Heist-op-den-Berg',
    'Lier', 'Mechelen', 'Nijlen', 'Putte', 'Puurs-Sint-Amands',
    'Sint-Katelijne-Waver', 'Willebroek', 'Zemst'
]
#lijst gemeenten overzetten in upper case zodat ik kan filteren
gemeenten_mechelen = [gemeente.upper() for gemeente in gemeenten_mechelen]

df_mechelen = df[df['Naam_NL'].isin(gemeenten_mechelen)].copy()
print(df_mechelen['Naam_NL'].unique())

In [ ]:
df_mechelen_no_fill = df_mechelen.copy()

In [ ]:
result_mechelen = inspect_df(df_mechelen)
visual_missing_percentage(result_mechelen)

In [ ]:
print(result_mechelen["missing_values"])

**Heatmap aantal  missende mediaanprijzen per periode en jaar**

Het percentage missing values ligt nu onder de 40%.

Ik ga nakijken in welke periodes en welke jaren de missing values liggen.

Ik roep dezelfde heatmap op via de functie.

Ik roep het aantal missing values per periode type op via de functie.

In [ ]:
heatmap_missing_values_period_year(df_mechelen)

In [ ]:
missing_per_periode_type_mechelen = missing_per_periode_type(df_mechelen)   
print(missing_per_periode_type_mechelen)

**Extra check**

Ik bekijk nog twee groeperingen of na te gaan of er geen andere duidelijke patronen zijn.

In [ ]:
missing_per_group = df_mechelen.groupby(['Periode', 'Woningtype_NL'])[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].apply(lambda x: x.isnull().sum())
#Aantal missende waarden per gemeente
missing_per_gemeente = df_mechelen.groupby('Naam_NL')[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].apply(lambda x: x.isnull().sum())

display(missing_per_gemeente)
display(missing_per_group)

**Eerste toepassing van forward_fill en backward_fill**

Ik pas de methode eerst toe op de kwartalen omdat de meeste missende waardes zich in de kwartalen bevinden

Ik werk binnen kwartalen van hetzelfde jaar

Periodes zoals semester en jaar worden niet aangepast

Ik vul de data per gemeente, jaar en woningtype

Hierna bekijk ik hoeveel missende waardes er nog over zijn --> 262


In [ ]:
#Kwartalen uitfilteren
df_mechelen_kwartaal = df_mechelen[df_mechelen["periode_type"] == "Kwartaal"].copy()
#Kwartalen opvullen
df_mechelen_kwartaal.loc[:,['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']] = (
    df_mechelen_kwartaal.groupby(['Naam_NL', 'Jaar', 'Woningtype_NL'])[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].transform(lambda group: group.ffill().bfill()))
#Dataframe updaten met nieuwe waardes
df_mechelen.update(df_mechelen_kwartaal)
#Nagaan hoeveel missende waardes er nog over zijn na de aanpassing
print(df_mechelen.isnull().sum())



**Opnieuw missing per periode type bekijen**

Nog niet alle kwartalen zijn gevuld. Dit komt omdat er voor sommige kwartalen geen waardes zijn om mee te vullen.




In [ ]:
missing_per_periode_type_mechelen_na_kwartalen1 = missing_per_periode_type(df_mechelen)
print(missing_per_periode_type_mechelen_na_kwartalen1)


**Nagaan of er bepaalde woningtypes zijn waarvoor er geen kwartaal is doorgegeven**

Sommige gemeentes hebben voor geen enkel kwartaal data geregistreerd bij bepaalde woningtypes.
Ik indentificeer welke gemeente, woningtype en jaar géén enkel kwartaal bevatten.

In [ ]:
# Groepeer per gemeente, woningtype en jaar en tel de missende kwartalen
missing_per_groep = df_mechelen_kwartaal.groupby(["Naam_NL", "Jaar", "Woningtype_NL"])[["Prijs_25e_Percentiel", "Prijs_50e_Percentiel", "Prijs_75e_Percentiel"]].apply(lambda x: x.isnull().sum())
missing_per_groep.head()
# Toon alleen de groepen waar alle waarden ontbreken
totaal_missing = missing_per_groep[(missing_per_groep == 4).all(axis=1)]
totaal_missing.head()

# Bekijk de eerste resultaten
totaal_missing.index.unique()
# df_mechelen.to_excel("df_mechelen.xlsx", index=False)


**Extra check**

We zien dat bv voor duffel effectief voor appartementen voor bepaalde jaren de 4 kwartalen niet zijn ingevuld.

In [ ]:
df_mechelen[(df_mechelen['Naam_NL'] == "DUFFEL") & (df["Woningtype_NL"] == "Appartementen")]


**Analyse van ontbrekende waarden per gemeente en woningtype**

Ik onderzoek het percentage ontbrekende waarden per combinatie van gemeente en woningtype.

Omdat ik werk met drie prijspercentielen (25e, 50e en 75e), bereken ik het gemiddelde missingspercentage om een algemeen beeld te krijgen van de datavolledigheid.

Ik focus op combinaties waarbij meer dan 50% van de waarden ontbreekt, omdat deze onvoldoende betrouwbaar zijn voor verdere analyse.

In totaal vond ik 5 van zulke combinaties, waarvan de meeste betrekking hebben op appartementen in 5 verschillende gemeenten.

Deze rijen nam ik niet meer mee in verdere analyse

In [ ]:
# Bereken het percentage missende waarden per gemeente en woningtype
missing_percentages = df_mechelen.groupby(['Naam_NL', 'Woningtype_NL'])[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].apply(lambda x: x.isnull().mean() * 100)
#print(missing_percentages)
# Neem het gemiddelde missende percentage over de drie prijspercentielen om een samenvattende score per combo van gemeente en woningtype.
missing_percentages['Gemiddeld_Missing_%'] = missing_percentages.mean(axis=1)
#print(missing_percentages)
# Filter alleen de combinaties waar meer dan 50% van de waarden ontbreekt
gemeenten_met_veel_missende_data = missing_percentages[missing_percentages['Gemiddeld_Missing_%'] > 50].reset_index()

# Toon alleen de relevante kolommen
gemeenten_met_veel_missende_data = gemeenten_met_veel_missende_data[['Naam_NL', 'Woningtype_NL', 'Gemiddeld_Missing_%']]
# Sorteer op grootste percentage eerst
gemeenten_met_veel_missende_data = gemeenten_met_veel_missende_data.sort_values(by='Gemiddeld_Missing_%', ascending=False)

print(gemeenten_met_veel_missende_data)




**Verwijderen van deze combinaties van gemeente en woningtype met meer dan 50% missing values**

Ik haal de combinaties van gemeente en woningtype op met meer dan 50% ontbrekende prijswaarden.

Daarna doe ik een left merge op df_mechelen om deze combinaties te markeren via de _merge-kolom.

Ik filter enkel de rijen met "left_only" om de ongewenste combinaties uit de dataset te verwijderen.

De _merge-kolom verwijder ik opnieuw.

Tot slot bereken ik hoeveel rijen verwijderd zijn en controleer ik hoeveel ontbrekende waarden er nog overblijven per periode.



In [ ]:
#de gemeenten met te veel missende data verwijderen.

#Haal de gemeenten en woningtypes op die ik wil verwijderen
gemeenten_woningtypes_te_verwijderen= gemeenten_met_veel_missende_data[['Naam_NL', "Woningtype_NL"]]

#Verwijder de '_merge' kolom als die al bestaat
if '_merge' in df_mechelen.columns:
    df_mechelen = df_mechelen.drop(columns=['_merge'])

# Merge de dataset om te zien welke rijen in de verwijderlijst zitten.
# Door indicator=True toe te voegen, ontstaat er een nieuwe kolom _merge, waarmee ik kan aanduiden welke rijen enkel in de oorspronkelijke dataset voorkomen.
df_mechelen = df_mechelen.merge(
    gemeenten_woningtypes_te_verwijderen, 
    on=['Naam_NL', 'Woningtype_NL'], 
    how='left', 
    indicator=True
)

#filteren op 'left_only' om de ongewenste combinaties te verwijderen + kopie nemen
df_mechelen_filterd = df_mechelen[df_mechelen["_merge"]== "left_only"].copy()

#ik verwijder de merge kolom want die is niet meer nodig
df_mechelen_filterd.drop(columns = ["_merge"], inplace = True)

#Berekenen van aantal verwijderde rijen
aantal_verwijderde_rijen = len(df_mechelen)- len(df_mechelen_filterd)
print(f"aantal verwijderde rijen = {aantal_verwijderde_rijen}")

#opnieuw missig values bekijken
missing_per_period_type = missing_per_periode_type(df_mechelen_filterd)
print(missing_per_period_type)

#df_mechelen_filterd.to_excel("df_mechelen_filterd.xlsx", index=False)

#opgeschoonde versie maken van df_mechelen
df_mechelen = df_mechelen_filterd

**Ontbrekede semesterwaarden opvullen**

Er zijn 10 ontbrekende semesterwaarden per prijspercentiel.

Eerst vullen we deze op met forward fill en backward fill.
    
Vervolgens zijn er nog 3 ontbrekende waarden over.
    
Deze verwijderen we.

In [ ]:
#Vervolgens de missende waarden in de semester periode aanpakken.
#filter enkel de rijen met 'Semester' als periode_type en maak een kopie
df_mechelen_semester = df_mechelen[df_mechelen["periode_type"] == "Semester"].copy()

#Vul de missende waarden op in de percentielkolommen per combinatie van gemeente, jaar en woningtype met ffill en bfill
df_mechelen_semester.loc[:,['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']] = (
    df_mechelen_semester.groupby(['Naam_NL', 'Jaar', 'Woningtype_NL'])[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].transform(lambda group: group.ffill().bfill()))

#Werk de originele dataframe bij met de aangepaste semesterdata
df_mechelen.update(df_mechelen_semester)

#Bereken het aantal missende waarden per periode_type na de opvulling
missing_per_period_type = missing_per_periode_type(df_mechelen)
print(missing_per_period_type)

#Filter de overgebleven semester-rijen waarin nog steeds minstens één NaN staat
df_mechelen_semester_missing = df_mechelen[
    (df_mechelen["periode_type"] == "Semester") & 
    (df_mechelen[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].isnull().any(axis=1))
]

#Verwijder deze resterende onvolledige semester-rijen uit de dataset
df_mechelen= df_mechelen.drop(df_mechelen_semester_missing.index)

#opnieuw controle op de missing values
missing_values_after = missing_per_periode_type(df_mechelen)
print("Aantal overgebleven missende waarden per periode type na verwijderen:")
print(missing_values_after)













**Invullen van ontbrekende kwartaalwaarden**

De ontbrekende kwartaalwaarden (Q1, Q2, Q3 en Q4) werden aangevuld op basis van de bijhorende semesterwaarden (S1 en S2).

Aangezien het gaat om mediaanprijzen per periode, heb ik ervoor gekozen om de waarde van het semester rechtstreeks toe te wijzen aan de bijhorende kwartalen.

De mediaanprijs van een semester is de prijs die in het midden ligt van alle verkopen in die 6 maanden.

We weten niet wat de exacte mediaanprijs was in elk kwartaal apart (Q1 en Q2), dus gebruiken we de mediaan van het volledige semester als een schatting.


In [ ]:
#Kolommen die we willen invullen
kolommen = ['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']

#Vul Q1 en Q2 in met S1 (als die bestaat)
for col in kolommen:
    df_mechelen.loc[df_mechelen['Periode'].isin(['Q1', 'Q2']), col] = df_mechelen.groupby(['Naam_NL', 'Jaar', 'Woningtype_NL'])[col].transform(
        lambda x: x.fillna(x[df_mechelen['Periode'] == 'S1'].values[0]) if len(x[df_mechelen['Periode'] == 'S1'].values) > 0 else x
    )

#Vul Q3 en Q4 in met S2 (als die bestaat)
for col in kolommen:
    df_mechelen.loc[df_mechelen['Periode'].isin(['Q3', 'Q4']), col] = df_mechelen.groupby(['Naam_NL', 'Jaar', 'Woningtype_NL'])[col].transform(
        lambda x: x.fillna(x[df_mechelen['Periode'] == 'S2'].values[0]) if len(x[df_mechelen['Periode'] == 'S2'].values) > 0 else x
    )

#Controleer of er nog missende waarden overblijven
missing_values_after = df_mechelen[kolommen].isnull().sum()
print("Aantal overgebleven missende waarden per kolom na invullen:")
print(missing_values_after)




**De 14 rijen met missende waardes verwijderen**

Ik verwijder eerst alle rijen met jaar 2024 en periode Q4 want we zagen eerder dat die volledig nul was.

vervolgens controleer ik weer de rijen met missende waardes. Dit zijn er nog steeds 14.

Ik verwijder deze volledig uit de dataframe.

In [ ]:
#Verwijder alleen de rijen waar 'Jaar' 2024 is en 'Periode' Q4
df_mechelen = df_mechelen[~((df_mechelen['Jaar'] == 2024) & (df_mechelen['Periode'] == 'Q4'))]

#Controleer hoeveel rijen er nog steeds missende waarden hebben
df_missing = df_mechelen[df_mechelen[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].isnull().any(axis=1)]

display(df_missing)

**Verwijder rijen uit 2024 die nog missende waardes bevatten (14)**

In [ ]:
#Verwijder rijen uit 2024 die nog missende waarden bevatten
df_mechelen = df_mechelen.dropna(subset=['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel'])

#Controleer of er nog missende waarden overblijven
missing_values_after = df_mechelen[['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']].isnull().sum()
print("Aantal overgebleven missende waarden per kolom na verwijderen probleemrijen:")
print(missing_values_after)


**Controle na het verwijderen**

In [ ]:
#Tel het aantal missende waarden per kolom in de volledige dataset
missing_values_final = df_mechelen.isnull().sum()

#Filter alleen de kolommen waar nog missende waarden zijn
missing_values_final = missing_values_final[missing_values_final > 0]

#Resultaat weergeven
if missing_values_final.empty:
    print("Er zijn geen missende waarden meer in de dataset!")
else:
    print("Er zijn nog steeds missende waarden in de volgende kolommen:")
    print(missing_values_final)

**Ik filter de dataset op de 3 woningtypes die van belang zijn voor analyse**

In [ ]:
df_mechelen_clean = df_mechelen[df_mechelen['Woningtype_NL'] != 'Alle huizen met 2, 3, 4 of meer gevels (excl. appartementen)']


**Eerste algemeen plot om +- na te gaan dat de data goed is opgeschoond**

In [ ]:
#Groepeer de dataset per jaar en woningtype, neem het gemiddelde van de mediaanprijzen
df_mechelen_jaar = df_mechelen_clean[df_mechelen_clean['periode_type'] == 'Jaar']
df_plot = df_mechelen_jaar.groupby(['Jaar', 'Woningtype_NL'])['Prijs_50e_Percentiel'].mean().reset_index()

#Maak een figuur
fig, ax = plt.subplots(figsize=(20, 10))

#Lijnplot voor elk woningtype
for woningtype in df_plot['Woningtype_NL'].unique():
    subset = df_plot[df_plot['Woningtype_NL'] == woningtype]
    ax.plot(subset['Jaar'], subset['Prijs_50e_Percentiel'], marker='o', linestyle='-', label=woningtype)
    
    #Voeg labels toe boven elke marker
    for x, y in zip(subset['Jaar'], subset['Prijs_50e_Percentiel']):
        ax.text(x, y, f"{int(y)}", ha='center', va='bottom', fontsize=15, color='black')


ax.set_xlabel("Jaar")
ax.set_ylabel("Gemiddelde Mediaanprijs (€)")
ax.set_title("Gemiddelde Mediaanprijs per Woningtype per Jaar")
ax.set_xticks(sorted(df_plot['Jaar'].unique()))  #Enkel hele jaartallen
ax.grid(True)
ax.legend(title="Woningtype", loc='upper left', bbox_to_anchor=(1.02, 1))
plt.savefig('plots/lijnplot Gemiddelde Mediaanprijs per Woningtype per Jaar', dpi=300, bbox_inches='tight')
plt.show()



**Controle op outliers na opschoning**

Na de opschoning van de dataset via forward en backward fill binnen combinatie van jaar, woningtype en gemeente, controleer ik opnieuw de outliers.

ik bekijk de shape van de outliers via de functie check_outliers, om te bepalen hoeveel rijen nog als uitschieter worden beschouwd.

Na de opschoning bleven er nog slechts 2 rijen over die als outlier werden gedetecteerd op de kolom Prijs_50e_Percentiel.

De conclusie is dat de resterende outliers realistisch en representatief zijn voor uitzonderlijke maar legitieme woningprijzen (bv. luxewoningen, grote panden, nieuwbouw). Daarom heb ik deze behouden in de dataset.

Onderstaande boxplots tonen ter vergelijking de verdeling van Prijs_50e_Percentiel voor en na opschoning, wat duidelijk het effect van de datacleaning illustreert.



In [ ]:
print(check_outliers(df_mechelen_clean, 'Prijs_25e_Percentiel').shape)
print(check_outliers(df_mechelen_clean, 'Prijs_50e_Percentiel').shape)
print(check_outliers(df_mechelen_clean, 'Prijs_75e_Percentiel').shape)



In [ ]:
create_boxplot(df,'Prijs_50e_Percentiel')

In [ ]:
create_boxplot(df_mechelen_clean,'Prijs_50e_Percentiel')

**Plot om de evolutie van de gemiddelde mediaanprijzen (Prijs_50e_Percentiel) per woningtype over de jaren te visualiseren.**

Ik heb reeds een dataframe df_plot, waarin reeds per jaar en per woningtype het gemiddelde van de kolom Prijs_50e_Percentiel is berekend.

Sorteren van df_plot op Woningtype_NL en Jaar

Ik voeg een nieuwe kolom YOY toe met de procentuele evolutie per woningtype, berekend op het gemiddelde van de Prijs_50e_Percentiel

Deze YOY-berekening toont hoe sterk de gemiddelde mediaanprijzen per woningtype zijn gestegen of gedaald ten opzichte van het voorgaande jaar.


In [ ]:
df_sorted_per_woningtype_jaar = df_plot.sort_values(['Woningtype_NL', 'Jaar']).reset_index(drop = True)
df_sorted_per_woningtype_jaar['YOY'] = (df_sorted_per_woningtype_jaar.groupby('Woningtype_NL')['Prijs_50e_Percentiel'].pct_change()*100).round(2)
df_sorted_per_woningtype_jaar['YOY_verschil'] = df_sorted_per_woningtype_jaar.groupby('Woningtype_NL')['YOY'].diff().round(2)
df_sorted_per_woningtype_jaar

**Plot Hoe snel stijgt of daalt de gemiddelde mediaanprijs van elk woningtype per jaar?**

Hoeveel % stijgt of daalt de gemiddelde Prijs_50e_Percentiel per woningtype t.o.v. het vorige jaar?

Appartementen tonen een sterk wisselend verloop:
een forse daling in 2021, herstel in 2022, en opnieuw daling in 2023.

Huizen met 2 of 3 gevels zijn het meest stabiel:
geen extreme uitschieters, enkel kleine fluctuaties.

Huizen met 4 of meer gevels (open bebouwing) kenden eerst een stijging, maar sinds 2022 twee jaar op rij daling, met in 2023 een daling van 9,38%.





In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

kleuren = {}
#Voor elke woningtype + groep van jaren
for woningtype, groep in df_sorted_per_woningtype_jaar.groupby('Woningtype_NL'):
    #verwijder de nan_waarden want voor 2019 zijn er geen waardes YOY
    groep = groep.dropna(subset=['YOY'])
    #Plot een lijn voor dit woningtype en bewaar het lijnobject om de kleur later te kunnen gebruiken (unpacking door gebruik van ,)
    lijn, = ax.plot(groep['Jaar'], groep['YOY'], marker='o', label=woningtype)
    #haal de kleur op per lijn om later te gebruiken
    kleur = lijn.get_color()
    kleuren[woningtype] = kleur
    
    #voor elke index en rij in groep (df met alle jaren voor een woningtype)
    #itterrows gaat door elke rij in de df en geeft een tuple terug van index en de rij zelf als een series opbject
    for i, row in groep.iterrows():
        #maak een label met yoy%
        label = f"{row['YOY']}%"
        #haal het verschil tov PY op
        yoy_verschil = row['YOY_verschil']
        #als yoy_verschil niet nul is positief symbool als yoy_verschil positief is, negatief symbool als yoy_verschil negatief is
        if pd.notna(yoy_verschil):
            symbool = '↑' if yoy_verschil > 0 else '↓'
            #Voeg het symbool en de absolute waarde van het verschil toe op een nieuwe regel. het symbool geeft de richting al aan vandaar absolute waarde
            label += f"\n{symbool} {abs(yoy_verschil)}%"
        #Bepaal of de tekst links of rechts uitgelijnd wordt, afhankelijk van even/oneven jaartal
        ha = 'left' if row['Jaar'] % 2 == 0 else 'right'
        #Zet de tekst op de plot, iets boven het punt op de lijn
        ax.text(
            row['Jaar'],
            row['YOY'] + 0.5,
            label,
            fontsize=12,
            color = kleur,
            ha=ha,
            va='top'
        )

        
#teken een nullijn
ax.axhline(0, color='grey', linestyle='--', linewidth=1)

#Titel en as-labels instellen
ax.set_title('Jaarlijkse YOY% evolutie in gemiddelde Prijs_50e_Percentiel per woningtype')
ax.set_xlabel('Jaar')
ax.set_ylabel('Evolutie t.o.v. vorig jaar (%)')

#Bepaal de unieke jaartallen waarvoor er een YOY-waarde bestaat
jaren_met_yoy = df_sorted_per_woningtype_jaar.dropna(subset=['YOY'])['Jaar'].unique()
#Gebruik die waardes als x_ticks
ax.set_xticks(sorted(jaren_met_yoy))

#toon een grid in de figuur
ax.grid(True)
#Maak een legende
ax.legend(title='Woningtype')
#spelen met de y_as voor leesbaarheid
ax.set_ylim(-4, 12)
#layout
plt.tight_layout()
ax.title.set_fontsize(16)
ax.xaxis.label.set_fontsize(14)
ax.yaxis.label.set_fontsize(14)
ax.tick_params(axis='both', labelsize=12)
ax.legend(title='Woningtype', fontsize=12, title_fontsize=13)
plt.savefig('plots/lijnplot Jaarlijkse YOY evolutie in gemiddelde Prijs_50e_Percentiel per woningtype', dpi=300, bbox_inches='tight')
plt.show()





**YOY-analyse voor meerdere prijspercentielen tegelijk**

Doel: Eenzelfde YOY-plot maken, maar voor de drie prijspercentielen tegelijk (25e, 50e, 75e percentiel).

Eerst een df maken met het gemiddelde per jaar en woningtype voor de drie prijspercentielen.

Daarna bereken ik voor elk van de drie kolommen:

De YOY (Year-over-Year) procentuele verandering per woningtype

Het verschil in YOY ten opzichte van het vorige jaar

**Conclusie**

De jaarlijkse prijsverandering (YOY) laat duidelijke verschillen zien tussen woningtypes en prijscategorieën:

Appartementen (blauw) tonen opvallende prijsschommelingen, vooral in de lagere percentielen (25e & 50e). In deze segmenten zie je sterke opwaartse én neerwaartse bewegingen, wat wijst op een hogere volatiliteit.

Open bebouwing (groen) kende tussen 2020 en 2022 steile prijsstijgingen, maar daalde in 2023 fors in het 25e en 50e percentiel. Ook hier is veel dynamiek zichtbaar, vooral aan de onder- en middenkant van de markt.

Gesloten en halfopen bebouwing (oranje) volgt een gelijkaardig, iets gematigder patroon, maar ook hier is de prijs in 2023 sterk teruggevallen in het 25e percentiel.

Het 75e percentiel toont in alle woningtypes de kleinste prijsschommelingen over de jaren heen. Dit suggereert dat de duurdere segmenten relatief stabieler zijn dan goedkopere woningen.

In [ ]:
df_mechelen_mean = df_mechelen_jaar.groupby(['Jaar', 'Woningtype_NL'])[['Prijs_25e_Percentiel','Prijs_50e_Percentiel','Prijs_75e_Percentiel']].mean().reset_index()
df_yoy = df_mechelen_mean.copy()
for kolom in ['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']:
    df_yoy[f'YOY_{kolom}'] = df_yoy.groupby('Woningtype_NL')[kolom].pct_change() * 100
    df_yoy[f'YOY_{kolom}'] = df_yoy[f'YOY_{kolom}'].round(2)
    df_yoy[f'YOY_verschil_{kolom}'] = df_yoy.groupby('Woningtype_NL')[f'YOY_{kolom}'].diff().round(2)
df_yoy
    

**Lijnplot evolutie gemiddelde mediaanprijs per jaar voor de drie prijspercentielen**

Ik maak een functie om het lijnplot met de evolutie over de jaren weer te geven voor de 3 prijspercentielen.

In [ ]:
def plot_yoy_alle_percentielen(df_yoy):
    prijskolommen = ['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']
    titels = ['25e percentiel', '50e percentiel (mediaan)', '75e percentiel']
    #maak een raster met 3 subplots, 3 rijen 1 kolom
    fig, axs = plt.subplots(3, 1, figsize=(14, 18))
    #voor elke index en prijspercentiel (25e, 50e, 75e) in prijskolommen
    for i, kolom in enumerate(prijskolommen):
        #bepaal de ax die je gaat gebruiken (ax = nummer index)
        ax = axs[i]
        #Maak de naam van de kolom om deze automatisch te kunnen selecteren. Dit zodat ik in 1 functie de 3 plots kan maken
        #En niet elk prijspercentiel apart moet aanroepen
        kol_yoy = f'YOY_{kolom}'

        #voor elk woningtype + groep van jaren
        for woningtype, groep in df_yoy.groupby('Woningtype_NL'):
            #verwijder jaren waarvoor geen waardes zijn (2019)
            groep = groep.dropna(subset=[kol_yoy])
            #Plot een lijn voor dit woningtype en bewaar het lijnobject om de kleur later te kunnen gebruiken (unpacking door gebruik van ,)
            lijn, = ax.plot(groep['Jaar'], groep[kol_yoy], marker='o', label=woningtype, color=kleuren.get(woningtype))
            #consistent kleurgebruik
            # kleuren['woningtype'] = kleur

        #Maak een nul-lijn
        ax.axhline(0, color='grey', linestyle='--', linewidth=1)
        #titels en assen instellen
        ax.set_title(f'Jaarlijkse % evolutie – {titels[i]}', fontsize=16, fontweight='bold')
        ax.set_ylabel('YoY (%)', fontsize=13)
        #schaal van de y-as voor de 3 plots gelijk maken
        ax.set_ylim(-6, 16)
        #Bepaal de unieke jaartallen waarvoor er een YOY-waarde bestaat
        jaren_met_yoy = df_yoy.dropna(subset= kol_yoy)['Jaar'].unique()
        #Gebruik die waardes als x_ticks zodat hij geen halve jaren weergeeft
        ax.set_xticks(sorted(jaren_met_yoy))
        ax.grid(True)
        ax.tick_params(axis='both', labelsize=12)

    #zet 1 label jaar op de x as van de laaste plot voor de 3 plots
    axs[-1].set_xlabel('Jaar', fontsize=13)
    #zet de legende rechts bovenaan
    axs[0].legend(title='Woningtype', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=11, title_fontsize=12)

    plt.tight_layout()
    plt.savefig('plots/lijnplots jaarlijkse evolutie alle percentielen.png', dpi=300, bbox_inches='tight')
    plt.show()


plot_yoy_alle_percentielen(df_yoy)

**Zelfde informatie in een barplot voor Prijs_50e_Percentiel**

In [ ]:
#Haal de unieke woningtypes op uit de kolom 'Woningtype_NL'
woningtypes = df_sorted_per_woningtype_jaar['Woningtype_NL'].unique()
#Maak een 1x3 raster van 3 lege grafieken met gedeelde y-as
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)


#Loop over elke index en elk woningtype
for i, woningtype in enumerate(woningtypes):
    #Kies de subplot die overeenkomt met de index
    ax = axs[i]
    #filter de data voor het juiste woningtype
    data = df_sorted_per_woningtype_jaar[df_sorted_per_woningtype_jaar['Woningtype_NL'] == woningtype]
    
    #maak de barplots en verzamel de bars in een lijst
    bars = ax.bar(data['Jaar'], data['YOY'], color=kleuren[woningtype])
    
    #combineer elke bar met de juiste rij in data met zip
    for bar, (_, row) in zip(bars, data.iterrows()):
        #haal de hoogte op van de rij
        hoogte = row['YOY']
        #maak een label met de waarde van YOY%
        label = f"{row['YOY']}%"
        #als het yoy_verschil niet nul is positief symbool als yoy_vershil > 0, negatief symbool als yoy_verschil < 0
        if pd.notna(row['YOY_verschil']):
            symbool = '↑' if row['YOY_verschil'] > 0 else '↓'
            #Voeg het symbool en de absolute waarde van het verschil toe op een nieuwe regel. het symbool geeft de richting al aan vandaar absolute waarde
            label += f"\n{symbool} {abs(row['YOY_verschil']):.2f}%"
        #label plaatsen 
        ax.text(
            #zet het label in het midden van de balk
            bar.get_x() + bar.get_width() / 2,
            #zet het label halverwege de hoogte van de balk
            hoogte / 2,
            label,
            #Horizontaal en verticaal gecentreerd
            ha='center',
            va='center',
            fontsize=9,
            color='black'
        )
    #teken een nullijn
    ax.axhline(0, color='grey', linestyle='--', linewidth=1)
    #Titel en as-labels instellen
    ax.set_title(woningtype, fontsize=11)
    ax.set_xlabel("Jaar")
    #zet het Ylabel enkel aan de linkerkant
    if i % 2 == 0:
        ax.set_ylabel("YOY (%)")
    ax.set_xticks(data['Jaar'])
#hoofdtitel
fig.suptitle("Procentuele verandering (YOY) in Prijs_50e_Percentiel per woningtype", fontsize=14)
#Optimaliseer de layout en laat bovenaan wat ruimte voor de hoofdtitel
#rect = [left, bottom, right, top] → 0 = begin figuur, 1 = einde figuur
#Top = 0.96 → laat 4% ruimte bovenaan
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('plots/barplot procentuele verandering (YOY) in Prijs_50e_Percentiel per woningtype', dpi=300, bbox_inches='tight')
plt.show()




**Plot: YOY-verandering in 25e, 50e en 75e percentiel per woningtype**

Ik zet het voorgaande plot om in een functie, zodat ik automatisch dit type visualisatie kan genereren voor elk van de drie prijspercentielen.

In [ ]:
def plot_yoy_bars_alle_percentielen_per_woningtype(df, kleuren):
    prijskolommen = ['Prijs_25e_Percentiel', 'Prijs_50e_Percentiel', 'Prijs_75e_Percentiel']
    titels = ['25e percentiel', '50e percentiel (mediaan)', '75e percentiel']
    woningtypes = df['Woningtype_NL'].unique()

    #Voor elke index en bijhorende waarde uit de lijst prijskolommen
    for i, kolom in enumerate(prijskolommen):
        #maak de naam van kolom yoy om later aan te roepen
        kol_yoy = f'YOY_{kolom}'
        #Maak de naam van kolom yoy_verschil om later aan te roepen. Dit zodat ik in 1 functie de 3 plots kan maken
        #En niet elk prijspercentiel apart moet aanroepen
        kol_yoy_verschil = f'YOY_verschil_{kolom}'

        #maak een raster van 1 rij en 3 kolommen voor de plots, deze delen een y-as
        fig, axs = plt.subplots(1, 3, figsize=(20, 6), sharey=True)

        #Voor elke index en woningtype in de lijst woningtypes
        for j, woningtype in enumerate(woningtypes):
            #Kies het plot waarop je data wilt plaatsen
            ax = axs[j]
            #selecteer de juiste data uit df aan de hand van het woningtype
            data = df[df['Woningtype_NL'] == woningtype]
            #sla de bars op als een object in een list bars
            bars = ax.bar(data['Jaar'],data[kol_yoy],color=kleuren[woningtype])

             #combineer elke bar met de juiste rij in data met zip
            for bar, (_, row) in zip(bars, data.iterrows()):
                #Haal de hoogte op van de rij
                hoogte = row[kol_yoy]
                #Maak een label met de waarde van YOY
                label = f"{hoogte:.2f}%"
                #als het yoy_verschil niet nul is positief symbool als yoy_vershil > 0, negatief symbool als yoy_verschil < 0
                if pd.notna(row[kol_yoy_verschil]):
                    symbool = '↑' if row[kol_yoy_verschil] > 0 else '↓'
                    #Voeg het symbool en de absolute waarde van het verschil toe op een nieuwe regel. het symbool geeft de richting al aan vandaar absolute waarde
                    label += f"\n{symbool} {abs(row[kol_yoy_verschil]):.2f}%"
                #Voeg het label toe in het midden van de staaf
                ax.text(
                    bar.get_x() + bar.get_width() / 2,
                    hoogte / 2,
                    label,
                    ha='center',
                    va='center',
                    fontsize=9,
                    color='black'
                )
            #maak een nullijn    
            ax.axhline(0, color='grey', linestyle='--', linewidth=1)
            #Titels en assen instellen
            ax.set_title(woningtype, fontsize=12)
            ax.set_xlabel("Jaar")
            ax.set_xticks(data['Jaar'])
            #Enkel aan de linkerkant het label YOY(%) plaatsen
            if j == 0:
                ax.set_ylabel("YOY (%)")
                
            ax.grid(True, axis='y', linestyle=':')
        #Zet een hoofdtitel boven de volledige figuur, afhankelijk van het percentiel
        fig.suptitle(f"Procentuele verandering (YOY) in {titels[i]} per woningtype",fontsize=15)
        plt.tight_layout(rect=[0, 0, 1, 0.94])
        plt.savefig(f'plots/barplot_YOY_{kolom}.png', dpi=300, bbox_inches='tight')
        plt.show()

plot_yoy_bars_alle_percentielen_per_woningtype(df_yoy, kleuren)

**PLot per gemeente yoy-verandering in Prijs_50e_Percentiel tussen Q3 2024 en Q3 2023.**

Ik wil gaan kijken per gemeente wat de yoy verandering is in de mediaan prijs per gemeente.

Ik wil daarbij naar de meeste recente data kijken want die is het meest actueel maar voor Q4 in 2024 hadden we geen waardes en deze rijen zijn gewist uit de dataset.

Daarom kies ik ervoor om de waardes van Q3 2024 te gaan vergelijken met Q3 2023. 

**Conclusie**

Gesloten + halfopen bebouwing (oranje):

Sterke prijsstijgingen in sommige gemeenten:

- Duffel +30.83%, Bornem +23.67%, Putte +12.94%

Sterke dalingen:

- Nijlen -17.93%, Mechelen -9.85%

Zeer grote verschillen tussen gemeenten, wat op een hoge volatiliteit wijst voor dit woningtype.

Open bebouwing (groen):
Grote negatieve veranderingen in:

- Puurs-Sint-Amands -21.66%, Sint-Katelijne-Waver -21.57%

Sterke stijgingen elders:

- Putte +22.65%, Bonheiden +10.42%

Dit woningtype toont extreme regionale verschillen met een aantal heel sterke uitschieters.

Appartementen (blauw):

Wisselend beeld:

- Sterke stijging in Willebroek +15.69%

- Daling in Mechelen -16.39%, Bornem -10.16%

Hier is de spreiding kleiner dan bij huizen, maar nog steeds duidelijke regionale verschillen zichtbaar. Er is ook minder data.

De evolutie van woningprijzen tussen 2023 en 2024 is erg afhankelijk van locatie én woningtype.

Vooral gesloten/halfopen woningen en open bebouwing tonen sterke stijgingen of dalingen, afhankelijk van de gemeente.

Appartementen zijn iets minder volatiel, maar ook daar zijn duidelijke dalingen en stijgingen merkbaar.

In [ ]:
#filter de dataset op Q3 in 2023 en 2024
df_mechelen_q3 = df_mechelen_clean[
    (df_mechelen_clean['Periode'] == 'Q3') &
    (df_mechelen_clean['Jaar'].isin([2023, 2024]))
]

#Groepeer per gemeente, woningtype en jaar en neem het gemiddelde van de kolom Prijs_50e_Percentiel of de mediaan
grouped_per_gemeente = df_mechelen_q3.groupby(['Naam_NL', 'Woningtype_NL', 'Jaar'])['Prijs_50e_Percentiel'].mean().reset_index()

#Maak de kolommen met YOY en YOY verschil waardes
grouped_per_gemeente['YOY'] = (grouped_per_gemeente.groupby(['Naam_NL','Woningtype_NL'])['Prijs_50e_Percentiel'].pct_change()*100).round(2)
grouped_per_gemeente['YOY_verschil'] = grouped_per_gemeente.groupby(['Naam_NL','Woningtype_NL'])['YOY'].diff().round(2)
grouped_per_gemeente
#Filter op enkel 2024
gemeente_yoy_2024 = grouped_per_gemeente[grouped_per_gemeente['Jaar'] == 2024].copy()

#bepaal de woningtypes en zet ze in een lijst
woningtypes_gemeente = gemeente_yoy_2024['Woningtype_NL'].unique()
#Maak een raster van 1 rij en 3 kolommen voor de plots
fig, axs = plt.subplots(1,3, figsize = (18,5))

#Voor elke index en woningtype in de  lijst woningtypes_gemeente
for i , woningtype in enumerate(woningtypes_gemeente):
    #Kies het plot waarop je data wilt plaatsen
    ax = axs[i]
    #selecteer de juiste data uit df aan de hand van het woningtype
    data = gemeente_yoy_2024[gemeente_yoy_2024['Woningtype_NL'] == woningtype].copy()
    #maak de barobjecten en slaag ze op in een lijst
    bars = ax.bar(data['Naam_NL'], data['YOY'], color = kleuren[woningtype])
    #koppel het juiste barobject aan de juiste data
    for bar, (_, row) in zip(bars, data.iterrows()):
        #Haal de hoogte op van de rij
        hoogte = row['YOY']
        #maak een label met de waarde van YOY
        label = f"{hoogte:+.2f}%"  

        # yoy_verschil = row['YOY_verschil']
        # if pd.notna(yoy_verschil):
        #     symbool = '↑' if yoy_verschil > 0 else '↓'
        #     label += f"\n{symbool} {abs(yoy_verschil):.2f}%"

        
        #Voeg het label toe in het midden van de bar
        ax.text(
            bar.get_x()+ bar.get_width() / 2,
            bar.get_height() / 2,
            label,
            ha = 'center',
            va = 'center',
            fontsize = 9,
            color = 'black')
    #titels en assen instellen
    ax.set_title(woningtype, fontsize = 11)
    ax.set_xticks(range(len(data['Naam_NL'])))
    ax.set_xticklabels(data['Naam_NL'], rotation=45, ha='right')
    ax.set_ylabel('YOY %')
#Plaats een subtitel boven de figuur
fig.suptitle("YOY-verandering in Prijs_50e_Percentiel per gemeente (Q3_2024 t.o.v. Q3_2023)", fontsize = 14)
plt.savefig('plots/barplot YOY-verandering in Prijs_50e_Percentiel per gemeente (Q3_2024 t.o.v. Q3_2023).png', dpi=300, bbox_inches='tight')
plt.tight_layout(rect = [0,0,1,0.95])
            
plt.show()

**Controle op impact van imputatie op mijn analyse**

Tijdens het visualiseren van de resultaten merkte ik dat er in sommige gemeentes en woningtypes grote jaarlijkse sprongen voorkwamen. Dat deed me afvragen of de imputatie van missende waarden (via forward- en backward fill, deels op basis van semesterwaarden) de data mogelijk verstoord heeft.

Om dit te controleren heb ik gefilterd op een paar specifieke gevallen waarvan ik vermoedde dat ze veel missende waarden bevatten, namelijk:

Gemeentes: 'DUFFEL' en 'BORNEM'

Woningtypes: 'Appartementen' en 'Huizen met 4 of meer gevels (open bebouwing)'

Zo kon ik na de fill de originele en ingevulde waarden naast elkaar leggen en de impact inschatten.

**Conclusie**

De vergelijking toont dat sommige kwartaalwaarden volledig ontbraken en ingevuld zijn met data uit semesters. Daardoor kunnen de jaarlijkse verschillen soms overdreven zijn.

Toch blijft de algemene trend per woningtype en gemeente betrouwbaar. De exacte percentages zijn dus met wat voorzichtigheid te bekijken, maar de richting en de verschillen blijven relevant voor de analyse.

In [ ]:
gemeentes = ['DUFFEL', 'BORNEM']
woningtypes = ['Appartementen', 'Huizen met 4 of meer gevels (open bebouwing)']
not_filled = df_mechelen_no_fill[
    (df_mechelen_no_fill['periode_type'] == 'Kwartaal') &
    (df_mechelen_no_fill['Naam_NL'].isin(gemeentes)) &
    (df_mechelen_no_fill['Woningtype_NL'].isin(woningtypes)) &
    (df_mechelen_no_fill['Jaar'].between(2019, 2024))
].copy()
filled = df_mechelen[(df_mechelen['periode_type'] == 'Kwartaal') &
    (df_mechelen['Naam_NL'].isin(gemeentes)) &
    (df_mechelen['Woningtype_NL'].isin(woningtypes)) &
    (df_mechelen['Jaar'].between(2019, 2024))
].copy()
#display(filled)

vergelijking = not_filled.merge(
    filled,
    on=['Jaar', 'Periode', 'Woningtype_NL', 'Naam_NL'],
    suffixes=('_not_filled', '_filled')
)[['Jaar', 'Periode', 'Woningtype_NL', 'Naam_NL', 'Prijs_50e_Percentiel_not_filled', 'Prijs_50e_Percentiel_filled']]
vergelijking

**Plot Aantal transacties per woningtype per jaar**

Deze visualisatie toont het aandeel van het aantal vastgoedtransacties per woningtype, gegroepeerd per jaar.

Door een stacked bar chart te gebruiken, zie ik niet alleen het totaal per jaar, maar ook hoe de verhoudingen tussen woningtypes evolueren over de tijd.


**Conclusie**

- Gesloten + halfopen bebouwing (oranje):

Vormt elk jaar ruim de helft van de transacties.

Sterk gedaald in 2020 (corona?), daarna terug op niveau in 2022.

- Appartementen (blauw):
Stabiel aandeel van ongeveer een vijfde tot een kwart.

Licht herstel sinds 2020, maar minder sterk dan bij huizen.

- Open bebouwing (groen):
Blijft redelijk stabiel, met licht dalende trend in absolute aantallen.

In 2023 opnieuw lager dan in 2022.

Huizen met 2 of 3 gevels zijn het populairst en vormen de ruggengraat van de markt.

Appartementen blijven belangrijk, maar nemen geen duidelijk groter aandeel op.

Open bebouwing wordt relatief minder vaak verkocht, mogelijk door schaarste, prijs of veranderende woonbehoeften.


In [ ]:
#Maak een dataframe met enkel het periode type jaar omdat we volledige jaren gaan bekijken
df_transacties = df_mechelen_clean[df_mechelen_clean['periode_type'] == 'Jaar']
#Groepeer per jaar en woningtype, en tel het aantal transacties per combinatie op
df_transacties_per_type = df_transacties.groupby(['Jaar', 'Woningtype_NL'])['Aantal_transacties'].sum().reset_index()
#pivot maken om een stacked barplot te kunnen maken
df_pivot = df_transacties_per_type.pivot(index='Jaar', columns='Woningtype_NL', values='Aantal_transacties')
display(df_pivot)

fig, ax = plt.subplots(figsize=(12, 6))

#Maak een array van 0 om de onderkant van elke bar te zijn , dus evenveel 0 als bars of jaren
#Deze array wordt per woningtype opgeteld zodat de lagen mooi op elkaar gestapeld worden
bottom = np.zeros(len(df_pivot))
#Maak een array met posities op de x-as, je gebruikt index om te beginnen vanaf positie 0
x = np.arange(len(df_pivot.index))

# Elke kolom (woningtype) als aparte laag stapelen
for woningtype in df_pivot.columns:
    #teken de bar op de x positie, met de waarde als hoogte en bottom als beginpunt
    ax.bar(x, df_pivot[woningtype], bottom=bottom, label=woningtype)
    #tel de waarde van het aantal transacties bij bottom zodat elke waarde boven de andere word gestapeld
    #Op deze manier schuift elke volgende laag omhoog, op basis van wat er al onder zit
    bottom += df_pivot[woningtype].values

#Titels en as instellen
ax.set_xticks(x)
ax.set_xticklabels(df_pivot.index)
ax.set_xlabel('Jaar')
ax.set_ylabel('Aantal transacties')
ax.set_title('Aandeel aantal transacties per woningtype per jaar')
ax.legend(title='Woningtype', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('plots/stackedbar Aandeel aantal transacties per woningtype per jaar.png',dpi=300, bbox_inches='tight')
plt.show()


**Plot transacties per jaar en transacties per gemeente**

ik wil nagaan welke woningtypes het meest verkocht worden per jaar en per gemeente

Vertrekken van df_transacties wat een dataframe is met alle rijen van het periode type jaar


**Conclusie**

- Aantal transacties per jaar (links):
Huizen met 2 of 3 gevels (oranje) blijven duidelijk het meest verkocht.

Sterke terugval in 2020.

Appartementen (blauw) stijgen licht sinds 2020.

Open bebouwing (groen) blijft relatief stabiel maar in kleinere aantallen.

- Aantal transacties per gemeente (rechts):
Mechelen springt eruit met veruit het grootste aantal transacties, voor alle woningtypes.

Daarna volgen Lier, Willebroek en Heist-op-den-Berg.

In Duffel, Bornem en Putte is het aantal transacties veel lager, maar wel gelijkmatiger verdeeld over woningtypes.

Huizen met 2 of 3 gevels domineren nog steeds de markt in elk jaar en in bijna elke gemeente.

De stad Mechelen vormt het hart van de vastgoedactiviteit in het arrondissement, gevolgd door enkele andere grotere gemeenten.

De spreiding per gemeente toont lokale verschillen in voorkeuren of aanbod per woningtype.







In [ ]:
# Unieke jaren en woningtypes
def plot_transacties_per_jaar(ax, df, kleuren):
    #groeperen per jaar en woningtype geef de sum van het aantal transacties
    df_transacties_per_type = df.groupby(['Jaar', 'Woningtype_NL'])['Aantal_transacties'].sum().reset_index()
    #haal de jaren op en sorteer deze. Deze komen op de x-as.
    jaren = sorted(df_transacties_per_type['Jaar'].unique())
    #Haal de woningtypes op
    woningtypes = df_transacties_per_type['Woningtype_NL'].unique()

    #Barbreedte en positie-instellingen
    bar_width = 0.15
    x = np.arange(len(jaren))

    #Voor elke index en woningtype in de  lijst woningtypes
    for i, woningtype in enumerate(woningtypes):
        #maak een lege lijst met y-waarden om te vullen met aantal_transactie waardes
        y_values = []
        #Voor elk jaar het aantal transacties ophalen en toevoegen aan y_values
        for jaar in jaren:
            waarde = df_transacties_per_type.loc[
                (df_transacties_per_type['Jaar'] == jaar) &
                (df_transacties_per_type['Woningtype_NL'] == woningtype),
                'Aantal_transacties'
            ]
            y_values.append(waarde.values[0]) #Gebruik values om van een serie object naar een numerieke waarde te gaan om te kunnen plotten op een bar
        #Maak de bars
        ax.bar(x + i * bar_width, y_values, width=bar_width, label=woningtype, color=kleuren.get(woningtype))

    # titels en assen instellen
    ax.set_xlabel('Jaar')
    ax.set_ylabel('Aantal transacties')
    ax.set_title('Aantal transacties per woningtype per jaar')
    ax.set_xticks(x + bar_width * (len(woningtypes) - 1) / 2)
    ax.set_xticklabels(jaren)
    # ax.legend(title='Woningtype', bbox_to_anchor=(1.05, 1), loc='upper left')

def plot_transacties_per_gemeente(ax, df, kleuren):
    #groeperen per gemeente en woningtype geef de sum van het aantal transacties over alle jaren
    df_transacties_per_gemeente = df.groupby(['Naam_NL', 'Woningtype_NL'])['Aantal_transacties'].sum().reset_index()
    #haal de gemeentes op en sorteer deze. Deze komen op de x-as.
    gemeenten = sorted(df_transacties_per_gemeente['Naam_NL'].unique())
    #Haal de woningtypes op
    woningtypes = df_transacties_per_gemeente['Woningtype_NL'].unique()
    
    #Barbreedte en positie-instellingen
    bar_width = 0.2
    x = np.arange(len(gemeenten))

    #Voor elke index en woningtype in de  lijst woningtypes
    for i , woningtype in enumerate(woningtypes):
        #maak een lege lijst met y-waarden om te vullen met aantal_transactie waardes
        y_values = []
        #Voor elke gemeente het aantal transacties ophalen en toevoegen aan y_values
        for gemeente in gemeenten:
            waarde = df_transacties_per_gemeente.loc[(df_transacties_per_gemeente['Naam_NL'] == gemeente) &
            (df_transacties_per_gemeente['Woningtype_NL'] == woningtype),
            'Aantal_transacties']
            y_values.append(waarde.values[0] if not waarde.empty else 0)
         #Maak de bars   
        ax.bar(x + i * bar_width, y_values, width=bar_width, label=woningtype, color=kleuren.get(woningtype))
    #titels en as instellen
    ax.set_xticks(x + bar_width * (len(woningtypes) - 1) / 2)
    ax.set_xticklabels(gemeenten, rotation=45, ha='right')
    ax.set_xlabel("Gemeente")
    ax.set_ylabel("Aantal transacties")
    ax.set_title("Aantal transacties per woningtype per gemeente")
    

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
plot_transacties_per_jaar(axs[0], df_transacties, kleuren)
plot_transacties_per_gemeente(axs[1], df_transacties, kleuren)
fig.legend(*axs[0].get_legend_handles_labels(),title='Woningtype',loc='lower center',bbox_to_anchor=(0.5, -0.15))
plt.tight_layout()
fig.savefig('plots/barplots aantal transacties per jaar en per gemeente.png', dpi=300, bbox_inches='tight')
plt.show()

**Prijs vs aantal transacties per woningtype**
    
Onderzoekt of duurdere woningen ook vaker (of net minder) verkocht worden

Inzichten uit scatterplot: Gemiddelde prijs vs aantal transacties

De scatterplot toont dat er geen positief verband is tussen de gemiddelde verkoopprijs en het aantal transacties per woningtype.

- Huizen met halfopen of gesloten bebouwing zijn veruit het populairst, met jaarlijks meer dan 2.000 transacties.
- Open bebouwing is het duurst, maar wordt het minst verkocht.
- Appartementen zijn gemiddeld het goedkoopst, maar ook hun transactievolume blijft relatief laag.

De middenklasse van woningen (halfopen/gesloten) blijkt dus het meest aantrekkelijk op de vastgoedmarkt in dit arrondissement, zowel qua prijs als verkoopaantallen.


In [ ]:
#Groepeer per jaar en woningtype en maak 2 kolommen met het gemiddelde en het aantal transacties
df_scatter = df_transacties.groupby(['Jaar', 'Woningtype_NL']).agg(gemiddelde_prijs = ('Prijs_50e_Percentiel', 'mean'), aantal_transacties = ('Aantal_transacties','sum')).reset_index()
display(df_scatter)

fig, ax  = plt.subplots(figsize = (12,6))
#Voor elk woningtype in de unieke woningtypes van df_scatter
for woningtype in df_scatter['Woningtype_NL'].unique():
    #filter de juiste data adh het woningtype
    subset = df_scatter[df_scatter['Woningtype_NL'] == woningtype]
    #Maak de scatter
    plt.scatter(subset['gemiddelde_prijs'], subset['aantal_transacties'],
              label = woningtype,s = 100)
    #for elke index in een range gelijk aan de lengte van de subset, zet een label naast het datapunt
    for i in range(len(subset)):
        plt.text(
        subset['gemiddelde_prijs'].iloc[i],
        subset['aantal_transacties'].iloc[i] + 50,
        str(subset['Jaar'].iloc[i]),
        fontsize=9
    )

plt.title('Gemiddelde prijs vs aantal transacties per woningtype')
plt.xlabel('Gemiddelde prijs (€)')
plt.ylabel('Aantal transacties')
plt.legend(title='Woningtype')
plt.grid(True)
plt.tight_layout()
plt.savefig('plots/scatterplot Gemiddelde prijs vs aantal transacties per woningtype.png',dpi=300, bbox_inches='tight')
plt.show()



**Gemiddelde mediaanprijs per jaar en woningtype**

Deze plot toont de gemiddelde mediaanprijs per jaar, uitgesplitst per woningtype (2019–2023). 

Het geeft een beeld van de prijsniveaus en hun evolutie.

**Conclusie**

Open bebouwing (groen) is veruit het duurst, met mediaanprijzen rond of boven de €400.000. De stijging is sterk tussen 2019 en 2022, met een kleine daling in 2023.

Huizen met 2 of 3 gevels (oranje) zitten in de middenmoot, met een consistente stijging tot ongeveer €315.000 in 2023.

Appartementen (blauw) zijn het goedkoopst, met een toename van ongeveer €225.000 naar €268.000.

Alle woningtypes tonen prijsstijging, vooral tussen 2019 en 2022.

In 2023 is er een lichte afvlakking of daling bij open bebouwing.

Er is een groot prijsverschil tussen verschillende woningtypes: een open bebouwing is gemiddeld bijna dubbel zo duur als een appartement.



In [ ]:
#Maak een nieuwe df met een duidelijke naam. Alle rijen van df_clean met een periodetype jaar.
df_mechelen_jaar = df_transacties.copy()
#Groepeer de dataframe per jaar en woningtype en neem het gemiddelde van de kolom prijs_50_percentiel.
df_mechelen_avg_jaar_woningtype = df_mechelen_jaar.groupby(['Jaar', 'Woningtype_NL']).agg(mediaanprijs = ('Prijs_50e_Percentiel', 'mean')).reset_index()
#bepaal de jaren en de woningtypes
jaren = sorted(df_mechelen_avg_jaar_woningtype['Jaar'].unique())
woningtypes = df_mechelen_avg_jaar_woningtype['Woningtype_NL'].unique()

#bar-instellingen
bar_width = 0.25
x = np.arange(len(jaren))


fig, ax = plt.subplots(figsize=(12, 6))

# Elke woningtype in woningtypes
for i, woningtype in enumerate(woningtypes):
    subset = df_mechelen_avg_jaar_woningtype[df_mechelen_avg_jaar_woningtype['Woningtype_NL'] == woningtype]
    prijzen = subset['mediaanprijs'].values
    ax.bar(x + i * bar_width, prijzen, width=bar_width, label=woningtype, color=kleuren.get(woningtype))

# titels en as instellingen
ax.set_xticks(x + bar_width)
ax.set_xticklabels(jaren)
ax.set_xlabel('Jaar')
ax.set_ylabel('Gemiddelde mediaanprijs (€)')
ax.set_title('Gemiddelde mediaanprijs per jaar en woningtype')
ax.legend(title='Woningtype')
ax.grid(axis='y', linestyle='--', alpha=0.5)

fig.tight_layout()
plt.savefig('plots/barplot Gemiddelde mediaanprijs per jaar en woningtype.png',dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
df_2023_jaar = df_mechelen_clean[
    (df_mechelen_clean['periode_type'] == 'Jaar') &
    (df_mechelen_clean['Jaar'] == 2023)
]

# Gemiddelde mediaanprijs per gemeente
gemprijs_per_gemeente = df_2023_jaar.groupby(['Woningtype_NL','Naam_NL'])['Prijs_50e_Percentiel'].mean().reset_index()
gemprijs_per_gemeente
#Voor elke woningtype in gemprijs_per_gemeente bepaal de goedkoopste en duurste top 5.
for woningtype in gemprijs_per_gemeente['Woningtype_NL'].unique():
    subset = gemprijs_per_gemeente[gemprijs_per_gemeente['Woningtype_NL'] == woningtype]
    top5_duurste = subset.sort_values(by='Prijs_50e_Percentiel', ascending=False).head(5)
    top5_goedkoopste = subset.sort_values(by='Prijs_50e_Percentiel', ascending=True).head(5)
    display(f'Top5 duurste gemeenten per {woningtype} in 2023')
    display(top5_duurste)
    display(f'Top5 goedkoopste gemeenten per {woningtype} in 2023')
    display(top5_goedkoopste)

In [ ]:
gdf = gpd.read_file('data/BELGIUM_-_Municipalities.geojson')
result_gdf = inspect_df(gdf)
print(gdf.columns)
print(gdf.crs)
gdf['CODE_INS'] = gdf['CODE_INS'].astype(int)
display(gdf.info())

In [ ]:
# df_mechelen_clean.columns
NIS_codes_mechelen_clean = df_mechelen_clean['NIS-code'].unique()
gdf_filtered = gdf[gdf['CODE_INS'].isin(NIS_codes_mechelen_clean)][['ADMUNAGE', 'CODE_INS', 'geometry']]
display(gdf_filtered)

#Verander de naam van de kolom met de gemeente
gdf_filtered = gdf_filtered.rename(columns={'ADMUNAGE': 'Naam_NL'})
display(gdf_filtered)

In [ ]:
df_mechelen_clean

**Kaart visual**

  Deze kaart toont de YOY-verandering (jaar-op-jaar) in de mediaanprijs per gemeente voor Q3 2024 t.o.v. Q3 2023, dus de procentuele stijging of daling van de mediane verkoopprijs per gemeente.

In [ ]:
#we vertrekken van dataframe gemeente_yoy_2024 die reeds per woningtype de yoy bevat voor alle gemeentes tussen Q3 2024 en Q3 2023.
#Deze groeperen tot het gemiddelde over alle woningtypes heen
df_kaart = gemeente_yoy_2024.groupby('Naam_NL').agg({'YOY': 'mean'}).reset_index()
#mergen met de geo data
merged_df_kaart = df_kaart.merge(gdf_filtered, on = 'Naam_NL')
#omzetten in de geo dataframe
merged_df_kaart = gpd.GeoDataFrame(merged_df_kaart, geometry='geometry', crs="EPSG:4326")
#nakijken of merged_df_kaart een geo dataframe is 
print(type(merged_df_kaart))

fig, ax = plt.subplots(figsize=(10, 8))

#plot Belgie als achtergrond
gdf.plot(ax=ax, color='lightgrey', edgecolor='white')

#plot de gemeenten
merged_df_kaart.plot(
    column='YOY',
    cmap='coolwarm',
    legend=True,
    edgecolor='black',
    linewidth=0.8,
    ax=ax
)

#Labels met naam van de gemeenten en de YOY waarde toevoegen
for idx, row in merged_df_kaart.iterrows():
    centroid = row['geometry'].centroid
    tekst = f"{row['Naam_NL']}\n{row['YOY']:.1f}%"
    ax.text(centroid.x, centroid.y, tekst,
            ha='center', va='center', fontsize=6, color='black')

#Door heel belgie te plotten is het gebied dat ik wil bekijken te klein dus ik zoom in op het gekleurde gebied
bounds = merged_df_kaart.total_bounds
ax.set_xlim(bounds[0] - 0.05, bounds[2] + 0.05)
ax.set_ylim(bounds[1] - 0.05, bounds[3] + 0.05)


ax.set_title('YOY-verandering in Prijs_50e_Percentiel per gemeente (Q3_2024 t.o.v. Q3_2023)', fontsize=13)
ax.axis('off')
plt.tight_layout()
plt.savefig('plots/kaart YOY-verandering in Prijs_50e_Percentiel per gemeente (Q3_2024 t.o.v. Q3_2023).png',dpi=300, bbox_inches='tight')
plt.show()



**Boxplot mediaanprijs per woningtype**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

df_mechelen_clean.boxplot(
    column='Prijs_50e_Percentiel',
    by='Woningtype_NL',
    ax=ax,
    grid=True,
    patch_artist=True,
    boxprops=dict(facecolor='lightblue', color='black'),
    medianprops=dict(color='red')
)

ax.set_title('Spreiding van Prijs_50e_Percentiel per woningtype')
ax.set_ylabel('Prijs (€)')
plt.suptitle('')
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()